In [1]:
import os
import polars as pl
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from tensorflow.keras import layers, models, optimizers, regularizers, callbacks



2024-11-22 20:40:14.895756: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-22 20:40:14.903068: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732279214.912301   13455 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732279214.915017   13455 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-22 20:40:14.924039: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
def get_generator_v3(dataframe, weights, feature_names, label_name, shuffle=True, batch_size=8192):
    def generator():
        indices = np.arange(len(dataframe))
        if shuffle:
            np.random.shuffle(indices)

        num_batches = len(indices) // batch_size + (1 if len(indices) % batch_size > 0 else 0)

        for i in range(num_batches):
            start_index = i * batch_size
            end_index = min((i + 1) * batch_size, len(dataframe))
            current_indices = indices[start_index:end_index]

            features = dataframe.iloc[current_indices][feature_names].values
            labels = dataframe.iloc[current_indices][label_name].values
            if weights is not None:
                weights_batch = weights.iloc[current_indices].values
            else:
                weights_batch = np.ones(len(labels), dtype=np.float32)

            yield features, labels.reshape(-1, 1), weights_batch

    return generator


In [10]:

def prepare_dataset(dataframe, weights, feature_names, label_name, batch_size=8192, shuffle=True):
    num_features = len(feature_names)

    output_signature = (
        tf.TensorSpec(shape=(None, num_features), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 1), dtype=tf.float32),
        tf.TensorSpec(shape=(None,), dtype=tf.float32)
    )

    dataset = tf.data.Dataset.from_generator(
        get_generator_v3(dataframe, weights, feature_names, label_name, shuffle, batch_size),
        output_signature=output_signature
    )

    return dataset

In [3]:
def create_model(input_dim, lr, weight_decay):
    # Create a sequential model
    model = models.Sequential()

    # Start with the first hidden layer with BatchNorm1d, SiLU, Dropout, and Linear layers
    # Input layer


    model.add(layers.Input(shape=(input_dim,)))


    model.add(layers.BatchNormalization())
    model.add(layers.Activation('swish'))
    model.add(layers.Dropout(0.1))  # Assuming dropouts[1] is valid
    model.add(layers.Dense(128, kernel_regularizer=regularizers.l2(weight_decay)))

    model.add(layers.BatchNormalization())
    model.add(layers.Activation('swish'))
    model.add(layers.Dropout(0.1))  # Assuming dropouts[1] is valid
    model.add(layers.Dense(64, kernel_regularizer=regularizers.l2(weight_decay)))

    # Output layer
    model.add(layers.Dense(1, activation='tanh'))

    # Compile model with Mean Squared Error loss
    # model.compile(optimizer=optimizers.Adam(learning_rate=lr), loss='mse', metrics=[WeightedR2()])
    model.compile(optimizer=optimizers.Adam(learning_rate=lr),
                  loss='mse',
                  metrics=[tf.keras.metrics.R2Score(class_aggregation='uniform_average')])
    return model


In [4]:

feature_names = [f"feature_{i:02d}" for i in range(79)] + [f"responder_{idx}_lag_1" for idx in range(9)]
label_name = 'responder_6'
weight_name = 'weight'


In [7]:
# start_id = 1400
# end_id = 1669
# folder_paths = [
#     f"/content/drive/MyDrive/JaneSt/CustomData_1/training/date_id={date_id}/*.parquet"
#     for date_id in range(start_id, end_id + 1)
# ]
# lazy_frames = [pl.scan_parquet(path) for path in folder_paths]
# combined_lazy_df = pl.concat(lazy_frames)
# df = combined_lazy_df.collect().to_pandas()



df = pl.scan_parquet(f"/home/zt/pyProjects/Optiver/JaneStreetMktPred/Lag_XGB/data/FOLD3").collect().to_pandas()
valid = pl.scan_parquet(f"/home/zt/pyProjects/Optiver/JaneStreetMktPred/Lag_XGB/data/validation.parquet").collect().to_pandas()




In [8]:
# df = pd.concat([df, valid]).reset_index(drop=True)
df[feature_names] = df[feature_names].ffill().fillna(0)
valid[feature_names] = valid[feature_names].ffill().fillna(0)

In [9]:
# symbol_cluster1 = [10, 0, 16, 1, 7, 35, 9, 33]

# df = df[df['symbol_id'].isin(symbol_cluster1)]
# valid = valid[valid['symbol_id'].isin(symbol_cluster1)]


In [12]:
X_train = df[ feature_names ]
y_train = df[ label_name ]
w_train = df[ "weight" ]
X_valid = valid[ feature_names ]
y_valid = valid[ label_name ]
w_valid = valid[ "weight" ]

# train_dataset = prepare_dataset(df, w_train)
# valid_dataset = prepare_dataset(valid, w_valid)

train_dataset = prepare_dataset(df, w_train, feature_names, label_name, batch_size=8129)
valid_dataset = prepare_dataset(valid, w_valid, feature_names, label_name, batch_size=8129)




W0000 00:00:1732279361.946292   13455 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [13]:

lr = 0.01
weight_decay = 5e-4

input_dim = df[feature_names].shape[1]
model = create_model(input_dim=input_dim, lr = lr, weight_decay=weight_decay)
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ batch_normalization             │ (None, 88)             │           352 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 88)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 88)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        11,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,577 (80.38 KB)

 Trainable params: 20,145 (78.69 KB)

 Non-trainable params: 432 (1.69 KB)

In [16]:
ca = [
    tf.keras.callbacks.EarlyStopping(monitor='val_r2_score', patience=50, mode='max'),
    tf.keras.callbacks.ModelCheckpoint(
        filepath='/home/zt/pyProjects/JaneSt/Team/scripts/George/models/1_base_model/tf_nn_bas_{epoch:02d}.keras',
        monitor='val_loss',
        save_best_only=False,  # Set to False to save every epoch
        save_freq='epoch'  # Save at the end of each epoch
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_r2_score',
        factor=0.1,
        patience=20,
        verbose=1,
        min_lr=1e-6
    )
]

In [17]:
model.fit(
    train_dataset.map(lambda x, y, w: (x, y, {'sample_weight': w})),
    epochs=100,
    validation_data=valid_dataset.map(lambda x, y, w: (x, y, {'sample_weight': w})),
    callbacks=ca
)

Epoch 1/100
   1670/Unknown 51s 30ms/step - loss: 1.6462 - r2_score: 0.0074

2024-11-22 20:45:20.105453: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


1672/1672 ━━━━━━━━━━━━━━━━━━━━ 55s 33ms/step - loss: 1.6462 - r2_score: 0.0074 - val_loss: 1.0741 - val_r2_score: 0.0035 - learning_rate: 0.0100
Epoch 2/100


2024-11-22 20:45:23.604254: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


1670/1672 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 1.6452 - r2_score: 0.0082

...

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


1672/1672 ━━━━━━━━━━━━━━━━━━━━ 54s 32ms/step - loss: 1.6427 - r2_score: 0.0095 - val_loss: 1.0732 - val_r2_score: 0.0037 - learning_rate: 0.0100
Epoch 8/100


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


1096/1672 ━━━━━━━━━━━━━━━━━━━━ 17s 30ms/step - loss: 1.6437 - r2_score: 0.0097

KeyboardInterrupt: 

In [ ]:

# Assume X_new is your new data you want to make predictions on
# This should be a NumPy array or a Tensor with shape (num_samples, num_features)
X_new = X_valid.to_numpy()  # Replace with actual data

# Make predictions
predictions = model.predict(X_new)
r2_metric = tf.keras.metrics.R2Score(class_aggregation='uniform_average')

if not isinstance(y_valid, np.ndarray):
    y_valid = y_valid.to_numpy()  # Convert to numpy array if it is a DataFrame

r2_metric.update_state(y_true=y_valid, y_pred=predictions)
r2_score_value = r2_metric.result().numpy()

print(f"R² Score on validation data: {r2_score_value}")


33820/33820 ━━━━━━━━━━━━━━━━━━━━ 39s 1ms/step
R² Score on validation data: 0.004446208477020264
